# Download images code
From: https://github.com/pereira-gha/activefire/

Images taken from satellite Landsat-8 and #poner la segunda api y de donde saca las imagenes#

In [1]:
#Library import
import os
import time
import pandas as pd
import rasterio

In [2]:
#Variables creation

#Link to the api of the Landsat 8 images
GC_L8 = 'https://storage.googleapis.com/gcp-public-data-landsat/LC08/01/'

MTL_EXTENSION = '_MTL.txt'
ext = '.TIF'

IN_DIR = r'../images/landsat_images/'
OUT_DIR = r'../images/landsat_images/tiff_images/'

if not os.path.exists(OUT_DIR): 
        os.makedirs(OUT_DIR) 

csv = "images_raster_Gianca.csv" #### Cambiar por el csv de cada quien
df = pd.read_csv(os.path.join(IN_DIR, csv), sep=';', index_col=0)
df.head()

,productId,entityId,acquisitionDate,cloudCover,processingLevel,path,row,min_lat,min_lon,max_lat,max_lon,download_url,PR,tipo
2073096,LC08_L1TP_118016_20200901_20200901_01_RT,LC81180162020245LGN00,2020-09-01 02:16:24.269623,43.82,L1TP,118,16,61.70824,132.79747,63.95174,137.63490,https://s3-us-west-2.amazonaws.com/landsat-pds...,118016,RT
2073103,LC08_L1TP_118017_20200901_20200901_01_RT,LC81180172020245LGN00,2020-09-01 02:16:48.152191,57.11,L1TP,118,17,60.32189,131.83264,62.55944,136.53466,https://s3-us-west-2.amazonaws.com/landsat-pds...,118017,RT
2073176,LC08_L1TP_134039_20200901_20200901_01_RT,LC81340392020245LGN00,2020-09-01 04:04:26.713446,14.99,L1TP,134,39,29.20954,95.59589,31.34188,98.06246,https://s3-us-west-2.amazonaws.com/landsat-pds...,134039,RT
2073382,LC08_L1TP_093086_20200902_20200902_01_RT,LC80930862020246LGN00,2020-09-02 00:09:48.053934,65.78,L1TP,93,86,-38.54002,143.01550,-36.40002,145.64833,https://s3-us-west-2.amazonaws.com/landsat-pds...,93086,RT
2073595,LC08_L1TP_045032_20200901_20200901_01_RT,LC80450322020245LGN00,2020-09-01 18:51:37.359774,13.64,L1TP,45,32,39.25627,-124.14699,41.38561,-121.42375,https://s3-us-west-2.amazonaws.com/landsat-pds...,45032,RT


In [3]:
df.describe()

,cloudCover,path,row,min_lat,min_lon,max_lat,max_lon,PR
count,24.000000,24.000000,24.000000,24.000000,24.000000,24.000000,24.000000,24.000000
mean,32.543750,108.375000,36.083333,32.833089,20.808863,35.040419,25.234304,108411.083333
std,31.356525,67.625447,28.489383,40.515472,108.822982,40.580568,109.040478,67638.228889
min,0.020000,7.000000,2.000000,-44.304530,-125.299230,-42.127910,-122.452080,7003.000000
25%,4.487500,46.000000,16.000000,18.386000,-74.074105,20.514645,-71.110305,46027.000000
50%,22.405000,117.000000,29.500000,42.811975,34.051210,44.953045,38.596035,117016.500000
75%,54.417500,139.500000,46.500000,61.683650,131.326337,63.938203,136.257767,139532.250000
max,100.000000,228.000000,90.000000,78.724180,144.561730,81.124190,147.228320,228090.000000


In [4]:
for ind, image_name in enumerate(df.productId.to_list()):
    start_time = time.time()
    print(ind+1, image_name)
    
    log = OUT_DIR + image_name + '.log'  
    if not os.path.exists(log):
        try:               
            path = image_name[10:13]
            row = image_name[13:16]      
            
            ########################### BQA ################################
            outnameBQA = OUT_DIR + image_name + '_' + 'BQA' + ext 
            if not os.path.exists(outnameBQA):
                link = GC_L8 + path + '/' + row + '/' + image_name + '/' + image_name + '_' + 'BQA' + ext
                with rasterio.open(link) as src:
                    profile = src.profile.copy()
                    BQA = src.read(1)
                
                with rasterio.open(outnameBQA, 'w', **profile) as dst:
                    dst.write_band(1, BQA.astype(rasterio.uint16))
                                   
            ########################### BANDS ################################
            outname = OUT_DIR + image_name + ext   
            if not os.path.exists(outname):
                link_bands = [GC_L8 + path + '/' + row + '/' + image_name + '/' + image_name + '_' + f'B{i}' + ext 
                            for i in [1,2,3,4,5,6,7,9,10,11]]
                
                bands = []
                profile = None
                for link in link_bands:
                    with rasterio.open(link) as src:
                        bands.append(src.read(1))
                        if profile is None:
                            profile = src.profile.copy()

                profile.update({
                    'dtype': rasterio.uint16,
                    'height': bands[0].shape[0],
                    'width': bands[0].shape[1],
                    'count': len(bands)
                })
                
                with rasterio.open(outname, 'w', **profile) as dst:
                    for i, band in enumerate(bands):
                        dst.write_band(i+1, band.astype(rasterio.uint16))

            # Move log file creation inside try block
            with open(log, 'w') as f:
                f.write('Done!')
            
            end_time = time.time()    
            print(f'Elapsed: {round((end_time-start_time)/60, 2)} min.\n')
            
        except Exception as e:
            print(f"Error processing {image_name}: {str(e)}")
            if os.path.exists(log):
                os.remove(log)  # Remove log file if processing failed

1 LC08_L1TP_118016_20200901_20200901_01_RT
2 LC08_L1TP_118017_20200901_20200901_01_RT
3 LC08_L1TP_134039_20200901_20200901_01_RT
Elapsed: 0.99 min.

4 LC08_L1TP_093086_20200902_20200902_01_RT
Elapsed: 0.94 min.

5 LC08_L1TP_045032_20200901_20200901_01_RT
Elapsed: 0.99 min.

6 LC08_L1GT_157012_20200902_20200902_01_RT
Elapsed: 1.06 min.

7 LC08_L1TP_007003_20200907_20200907_01_RT
Elapsed: 1.35 min.

8 LC08_L1TP_025033_20200905_20200905_01_RT
Elapsed: 0.98 min.

9 LC08_L1TP_041035_20200905_20200905_01_RT
Elapsed: 0.99 min.

10 LC08_L1TP_046021_20200908_20200908_01_RT
Elapsed: 0.97 min.

11 LC08_L1TP_046029_20200908_20200908_01_RT
Elapsed: 0.95 min.

12 LC08_L1TP_046031_20200908_20200908_01_RT
Elapsed: 0.89 min.

13 LC08_L1TP_092086_20200911_20200911_01_RT
Elapsed: 0.95 min.

14 LC08_L1TP_105069_20200906_20200906_01_RT
Elapsed: 0.98 min.

15 LC08_L1TP_117016_20200910_20200910_01_RT
Elapsed: 1.52 min.

16 LC08_L1TP_117017_20200910_20200910_01_RT
Elapsed: 0.93 min.

17 LC08_L1TP_119016_20200